In [1]:
# LSTM

#1. Load sample text file
import tensorflow as tf
path_to_file = tf.keras.utils.get_file("nietzsche.txt", 
                  origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")

In [2]:
#2. Read the text
text = open(path_to_file, "rb").read().decode(encoding='utf-8')
text = text.lower()

# Number of characters in the text
n_char = len(text)
print(f"Length of text: {n_char} characters")

Length of text: 600893 characters


In [3]:
#3. List of unique characters

vocab = sorted(set(text))
num_vocab = len(vocab)
num_vocab

57

In [4]:
#4. Create a mapping for each text
char_to_int = dict((c,i) for i, c in enumerate(vocab))

int_to_char = dict((i,c) for i, c in enumerate(vocab))

In [11]:
# 5. Prepare the dataset of input to output pairs
## "Hello" --> input: "Hell" output: "o"

seq_length = 40
dataX = []
dataY = []

for i in range(0, n_char-seq_length, 1):
    seq_in = text[i:i+seq_length] # Inout substring
    seq_out = text[i+seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
    
n_patterns = len(dataX)
print(f"Total patterns :{n_patterns}")

Total patterns :600853


In [12]:
len(dataX)

600853

In [13]:
#6. Preprocessing the input data
from tensorflow.keras.utils import to_categorical
import numpy as np

# Reshape X to be [Samples, time_steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))

# Normalize
X = X/float(num_vocab)

# One hot encode
y = to_categorical(dataY)

In [15]:
y[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0.], dtype=float32)

In [18]:
# 7. Build the model

from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.callbacks import ModelCheckpoint

model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy')

In [ ]:
#8. Train and save model

filepath = "LSTM_tutorial_model.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
hostory = model.fit(X, y, epochs=20, batch_size=128, callbacks=[checkpoint])

Epoch 1/20
4695/4695 [==============================] - ETA: 0s - loss: 2.8614
Epoch 00001: loss improved from inf to 2.86145, saving model to LSTM_tutorial_model.hdf5
4695/4695 [==============================] - 649s 138ms/step - loss: 2.8614
Epoch 2/20
4694/4695 [============================>.] - ETA: 0s - loss: 2.7033
Epoch 00002: loss improved from 2.86145 to 2.70331, saving model to LSTM_tutorial_model.hdf5
4695/4695 [==============================] - 703s 150ms/step - loss: 2.7033
Epoch 3/20
1348/4695 [=======>......................] - ETA: 7:29 - loss: 2.6442